In [1]:
import numpy as np
from DS_Planck_Unet import *

In [2]:
def gen_patches_centers(clusters, big_pixels, batch_size, search_nside=256, 
                        min_rad=2.0, mask_rad_ak=5/60, output_nside=2048):
    import healpy as hp
    import numpy as np
    from DS_healpix_fragmentation import pix2radec, draw_proper_circle_s, matr2dict
    
    small_pixels, df = pixels_with_clusters(clusters, big_pixels, search_nside, min_rad)
    centers = []
   
    ra, dec = pix2radec(nside=search_nside, ipix=small_pixels)
    all_coords = list(zip(ra, dec))
    del(ra)
    del(dec)
    del(small_pixels)
    
    while len(centers) < batch_size:
        idx = np.random.randint(len(all_coords))
        ra, dec = all_coords[idx]
        del(all_coords[idx])
       
        matr = gen_matr(ra, dec)
        mdict = matr2dict(matr)
        if matr.shape[0] != 64 or matr.shape[1] != 64:
            continue
        
        mask = draw_proper_circle_s(df['RA'], df['DEC'], mask_rad_ak, output_nside, 
                mdict, shape=matr.shape)
        if np.count_nonzero(mask) > 0:
            centers.append((ra, dec)) 
    return centers

In [ ]:
train_cen_coords = gen_patches_centers('/home/rt2122/Data/clusters/planck_z.csv', 
                                       train_pix, 90000)

In [4]:
val_cen_coords = gen_patches_centers('/home/rt2122/Data/clusters/planck_z.csv',
                                    val_pix, 10000, search_nside=128)

ValueError: low >= high